Presidio-anonymizer는 식별된 엔터티에서 수행할 임의의 작업을 허용할 수 있습니다. 이러한 작업은 람다 함수의 형태로 전달할 수 있습니다.

다음 예에서는 가짜 값을 사용하여 가명화를 수행합니다. 먼저 연산자를 살펴보겠습니다.

In [1]:
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig, EngineResult, RecognizerResult
from faker import Faker


fake = Faker()

# Create faker function (note that it has to receive a value)
def fake_name(x):
    return fake.name()


# Create custom operator for the PERSON entity
operators = {"PERSON": OperatorConfig("custom", {"lambda": fake_name})}

# Analyzer output
analyzer_results = [RecognizerResult(entity_type="PERSON", start=11, end=18, score=0.8)]

text_to_anonymize = "My name is Raphael and I like to fish."

anonymizer = AnonymizerEngine()

anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize, analyzer_results=analyzer_results, operators=operators
)

print(anonymized_results.text)


My name is Wendy Carr and I like to fish.


이것은 간단한 예이지만 다음은 고급 익명화 옵션에 대한 몇 가지 예입니다.

1. 성별을 식별하고 동일한 성별에서 임의의 값을 생성합니다(예: Laura -> Pam).

2. 날짜 패턴 식별 및 날짜 이동 수행(01-01-2020 -> 05-01-2020)

3. 나이와 버킷을 10년 단위로 식별(89 -> 80-90)

# 암호화 및 복호화

이 샘플은 Presidio Anonymizer 내장 기능을 사용하여 식별된 엔터티를 암호화하고 해독하는 방법을 보여줍니다.

암호화는 CBC 모드에서 AES 암호를 사용하며 암호화 및 암호 해독 모두에 대한 입력으로 암호화 키가 필요합니다.

# 가져오기 설정

In [2]:
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine
from presidio_anonymizer.entities import (
    RecognizerResult,
    OperatorResult,
    OperatorConfig,
)


# 암호화 키 정의 (암호화 및 복호화 모두)

In [8]:
crypto_key = "WmZq4t7w!z%C&F)J"

# Presidio Anonymizer : 암호화 

In [9]:
engine = AnonymizerEngine()

# Invoke the anonymize function with the text
# analyzer results (potentially coming from presidio-analyzer)
# and an 'encrypt' operator to get an encrypted anonymization output:
anonymize_result = engine.anonymize(
    text="My name is James Bond",
    analyzer_results=[
        RecognizerResult(entity_type="PERSON", start=11, end=21, score=0.8),
    ],
    operators={"PERSON" : OperatorConfig("encrypt", {"key" : crypto_key})},
)

anonymize_result

text: My name is LNT7NqECytEygKJq2YWmVAmtY+pZn3R7qmpvqB2PaJ4=
items:
[
    {'start': 11, 'end': 55, 'entity_type': 'PERSON', 'text': 'LNT7NqECytEygKJq2YWmVAmtY+pZn3R7qmpvqB2PaJ4=', 'operator': 'encrypt'}
]

출력에는 익명화된 텍스트와 암호화된 엔터티의 위치가 모두 포함됩니다.

이것은 전체 텍스트가 아닌 엔터티만 해독해야 하므로 유용합니다.

In [10]:
# Fetch the anonymized text from the result.
anonymized_text = anonymize_result.text

# Fetch the anonynized entities from the result.
anonymized_entities = anonymize_result.items

# Presidio Anonymizer: 복호화

In [11]:
# Initialize the engine:
engine = DeanonymizeEngine()

# Invoke the deanonymize function with the text, anonymizer results
# and a 'decrypt' operator to get the original text as output.
deanonymized_result = engine.deanonymize(
    text=anonymized_text,
    entities=anonymized_entities,
    operators={"DEFAULT" : OperatorConfig("decrypt", {"key" : crypto_key})},
)

deanonymized_result

text: My name is James Bond
items:
[
    {'start': 11, 'end': 21, 'entity_type': 'PERSON', 'text': 'James Bond', 'operator': 'decrypt'}
]

call Decrypt operator directly